In [49]:
import pandas as pd
import sys
from datetime import date, datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [50]:
sys.path.insert(0,"..")
bremen_trips = pd.read_csv("../data/processed/trips_weather.csv", index_col=0)
bremen_trips = bremen_trips.sort_values(by=['start_time'], ascending=True)
bremen_trips["start_time"] = pd.to_datetime(bremen_trips["start_time"])

In [51]:

for col in ['start_time']:
    bremen_trips['month'] = pd.DatetimeIndex(bremen_trips['start_time']).month
    bremen_trips['booking_date'] = bremen_trips.start_time.dt.date
    bremen_trips['weekdays'] = pd.DatetimeIndex(bremen_trips['start_time']).weekday

In [54]:
bremen_trips['duration_min'] = bremen_trips['duration_sec']/60
data_raw =bremen_trips

In [55]:
data_raw['hour'] = data_raw['start_time'].dt.hour
data_raw['min'] = data_raw['start_time'].dt.minute
data_raw['day_of_month'] = data_raw['start_time'].dt.day
data_raw.drop(axis = 1, columns = 'start_time', inplace=True)

In [57]:
for col in data_raw.columns:
    pct_missing = np.mean(data_raw[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

identification - 0.0%
end_time - 0.0%
duration_sec - 0.0%
start_lng - 0.0%
start_lat - 0.0%
end_lng - 0.0%
end_lat - 0.0%
start_place - 0.0%
end_place - 0.0%
start_plz - 0.0%
end_plz - 0.0%
month - 0.0%
booking_date - 0.0%
weekdays - 0.0%
duration_min - 0.0%
timestamp - 0.0%
temp_2m - 0.0%
humidity_2m - 0.0%
dew_point_2m - 0.0%
max_at_2m - 0.0%
mean_speed_h/s - 0.0%
direction_degree_x - 0.0%
max_m/s - 0.0%
min_mean_m/s - 0.0%
max_mean_m/s - 0.0%
direction_degree_y - 0.0%
min - 0.0%
hour - 0.0%
day_of_month - 0.0%


In [59]:
data_raw.drop(columns=["identification", "end_time", "end_lng", "end_lat", "end_place", "end_plz", "booking_date", "timestamp"], inplace=True)

In [61]:
data_raw.corrwith(data_raw['duration_min'])[:60]

duration_sec          1.000000
start_lng             0.006962
start_lat             0.039297
start_place           0.057302
start_plz             0.048741
month                -0.106360
weekdays              0.011312
duration_min          1.000000
temp_2m               0.017365
humidity_2m          -0.058349
dew_point_2m         -0.026082
max_at_2m             0.017693
mean_speed_h/s        0.021873
direction_degree_x   -0.000834
max_m/s               0.027060
min_mean_m/s          0.008624
max_mean_m/s          0.022586
direction_degree_y   -0.001909
min                  -0.004710
hour                 -0.028797
day_of_month          0.007346
dtype: float64

It is evident, that almost none of the features correlates to the trip duration. Thus we could conclude already that it is not possible to predict 

In [62]:
X = data_raw.drop(columns={'duration_min','duration_sec'})
#X = data_raw[['start_plz','month','start_lng','start_lat','hour']]
y = data_raw['duration_min']

In [64]:
from sklearn.model_selection import train_test_split


X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.60, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.60, shuffle=False)

In [65]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [66]:
poly = PolynomialFeatures(2)
X_train_poly = poly.fit_transform(X_train)

In [67]:
lin = LinearRegression()

In [68]:
lin.fit(X_train_poly, y_train)
lin.score(X_train_poly, y_train)

0.03328200584412799

In [69]:
X_val_poly = poly.fit_transform(X_val)
lin.score(X_val_poly,y_val)

-0.03630686139425232

In [70]:
from sklearn.ensemble import RandomForestRegressor


In [71]:
rf = RandomForestRegressor(criterion='mse', max_depth=5, n_estimators=512, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=512, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [72]:
y_pred = rf.predict(X_train)

In [73]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mean_absolute_error(y_train, y_pred)

64.92769523946832

In [136]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

parameters = {'max_depth':[2, 3, 4, 5, 7],
              'n_estimators':[1, 10, 25, 50, 100, 256, 512],
              'random_state':[42]}
    
def perform_grid_search(X_data, y_data):
    rf = RandomForestRegressor(criterion='mse')
    
    clf = GridSearchCV(rf, parameters, cv=4, scoring='r2', n_jobs=3)
    
    clf.fit(X_data, y_data)
    
    print(clf.cv_results_['mean_test_score'])
    
    return clf.best_params_['n_estimators'], clf.best_params_['max_depth']

In [137]:
# extract parameters
n_estimator, depth = perform_grid_search(X_train, y_train)
c_random_state = 42
print(n_estimator, depth, c_random_state)

[ 0.00948408  0.01434006  0.01197755  0.01549719  0.01703563  0.01710094
  0.01691182  0.01038098  0.02736511  0.02565955  0.02824473  0.03034348
  0.03091979  0.0300638  -0.02478662  0.02176968  0.01924592  0.02177784
  0.02471591  0.02553426  0.02558391 -0.0747195   0.01529336  0.00885957
  0.01301461  0.01588311  0.01541091  0.0157104  -0.07502949 -0.00744773
 -0.01977203 -0.01238211 -0.00838997 -0.00901166 -0.0069959 ]
256 3 42
